# Context Database preparation

## Importing essential libraries

In [ ]:
!pip install -q -U sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.0 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
!pip install -qqq -U --upgrade datasets hnswlib numpy pandas  torch PyPDF2 transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0

In [ ]:
import torch
import json
import os
import zipfile
import PyPDF2
import pandas as pd
from transformers import AutoTokenizer
import numpy as np
import hnswlib
from tqdm.auto import tqdm
from datasets import load_dataset

## Loading Document and Extracting their text
- Consitution of India pdf
- Formats of Legal Documents txt files
- central Act json format

In [ ]:
# Initialize list to store the extracted text
Text = []

# Open the PDF file in read-binary mode
with open('/content/COI...pdf', 'rb') as f:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(f)

    # Iterate through each page in the PDF after 31st page because of relevant content starts from there
    for page_num in range(31, len(pdf_reader.pages)):
        # Get the page object
        Text.append(''.join(pdf_reader.pages[page_num].extract_text().split('\n')[1:]))

In [ ]:
txtfiles = [os.path.join('/content/', files) for files in os.listdir('/content/') if files.endswith('.txt')]
for txtfile in txtfiles:
    with open(txtfile, 'r') as f:
        Text.append(f.read())

In [ ]:
!wget -O centralActs.zip https://zenodo.org/api/records/5088102/files-archive
!unzip -l centralActs.zip

--2024-05-24 12:42:20--  https://zenodo.org/api/records/5088102/files-archive
Resolving zenodo.org (zenodo.org)... 188.185.79.172, 188.184.98.238, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.185.79.172|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘centralActs.zip’

centralActs.zip         [     <=>            ]  12.69M  11.8MB/s    in 1.1s    

2024-05-24 12:42:22 (11.8 MB/s) - ‘centralActs.zip’ saved [13308510]

Archive:  centralActs.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
 13308320  2024-05-24 14:42   annotatedCentralActs.zip
---------                     -------
 13308320                     1 file


In [ ]:
os.makedirs('annotatedCentralActs', exist_ok=True)
def unzip_all(zip_file, destination):
    with zipfile.ZipFile(zip_file, 'r') as f:
        f.extractall(destination)
        os.remove(zip_file)

    for root, dirs, files in os.walk('/content/'):
        for f in files:
            if f.endswith('.zip'):
                file_path = os.path.join(root, f)
                unzip_all(file_path, root)
unzip_all('/content/centralActs.zip', '/content/annotatedCentralActs')

In [ ]:
files = [os.path.join('/content/annotatedCentralActs', File) for File in os.listdir('/content/annotatedCentralActs')]
print('Number of json files = ', len(files))

Number of json files =  858


In [ ]:
# Open and read the JSON file
with open(files[0], 'r') as file:
    print(json.load(file))

{'Act Title': 'THE SALE OF GOODS ACT, 1930', 'Act ID': 'ACT NO. 3 OF 1930', 'Enactment Date': '[15th March, 1930.]', 'Act Definition': {'0': 'An Act to define and amend the law relating to the sale of goods.', '1': 'WHEREAS it is expedient to define and amend the law relating to the sale of goods'}, 'Chapters': {'0': {'ID': 'CHAPTER I', 'Name': 'PRELIMINARY', 'Sections': {'Section 1.': {'heading': 'Short title, extent and commencement.', 'paragraphs': {'0': '(1) This Act may be called the 2*** Sale of Goods Act, 1930.', '1': '3[(2) It extends to the whole of India 4***.]', '2': '(3) It shall come into force on the 1st day of July, 1930.'}}, 'Section 2.': {'heading': 'Definitions.', 'paragraphs': {'0': {'text': 'In this Act, unless there is anything repugnant in the subject or context,—', 'contains': {'0': '(1) “buyer” means a person who buys or agrees to buy goods;', '1': '(2) “delivery” means voluntary transfer of possession from one person to another;', '2': '(3) goods are said to be

In [ ]:
df = []
for File in files:
    with open(File, 'r') as f:
        data = json.load(f)
    df.append(data)

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df.tail(1)

,Act Title,Act ID,Enactment Date,Act Definition,Chapters,Schedule,Annexure,Appendix,Forms,Footnotes,Sections,Parts
857,THE GEOGRAPHICAL INDICATIONS OF GOODS (REGISTR...,ACT NO. 48 OF 1999,"[30th December, 1999.]",{'0': 'An Act to provide for the registration ...,"{'0': {'ID': 'CHAPTER I', 'Name': 'PRELIMINARY...",{},{},{},{},"{'Page 4': {'0': '1. 15th September, 2003, vid...",NaN,NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 858 entries, 0 to 857
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Act Title       858 non-null    object
 1   Act ID          858 non-null    object
 2   Enactment Date  858 non-null    object
 3   Act Definition  854 non-null    object
 4   Chapters        366 non-null    object
 5   Schedule        858 non-null    object
 6   Annexure        858 non-null    object
 7   Appendix        858 non-null    object
 8   Forms           858 non-null    object
 9   Footnotes       858 non-null    object
 10  Sections        433 non-null    object
 11  Parts           59 non-null     object
dtypes: object(12)
memory usage: 80.6+ KB


In [ ]:
# appending all the acts in Text
for i, row in df.iterrows():
    try:
        Text.append(" ".join([row['Act ID'], row['Act Title'], ' '.join(list(row['Act Definition'].values())), row['Enactment Date']]))
    except:
        Text.append(" ".join(str(element) for element in df.iloc[i, :].tolist()))
print('Number of sentences for context = ', len(Text))

Number of sentences for context =  1275


## Initializing Hierarchical Navigating Small World Vector store

In [ ]:
db = hnswlib.Index(space = 'l2', dim = 768)

In [ ]:
db.init_index(max_elements = len(Text), M = 16, ef_construction = 200, random_seed = 100, allow_replace_deleted = False)

## Load Embedding Model

In [ ]:
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# load the retriever model from huggingface model hub
vector_generator = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)
vector_generator

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

## Creating and Storing vectors to Hiearichal Navigating Small World vector store

In [ ]:
%%time
data = [np.array(vector_generator.encode(Text[index]).tolist()) for index in tqdm(range(len(Text)))] # generate embeddings
db.add_items(np.array(data), range(len(Text)), num_threads = -1, replace_deleted = False) # add index to space

  0%|          | 0/1275 [00:00<?, ?it/s]

CPU times: user 15min 10s, sys: 46 s, total: 15min 56s
Wall time: 16min 22s


In [ ]:
def get_relevant_sentences(question: str, top_k: int) -> list:
    try:
        query = np.array(vector_generator.encode(question).tolist())
        # Query dataset, k - number of the closest elements (returns 2 numpy arrays)
        #labels, distances = db.knn_query(query, k = 4)
        #return [Text[label] for label in labels[0]]
        return [Text[label] for label in db.knn_query(query, top_k)[0][0]]
    except Exception as e:
        print({e})

In [ ]:
get_relevant_sentences("Freedom of Speech", 2)

['ACT NO. 69 OF 1971 THE PREVENTION OF INSULTS TO NATIONAL HONOUR ACT, 1971 An Act to prevent insults to national honour. [23rd December, 1971.]',
 'ACT NO. 15 OF 1977 THE PARLIAMENTARY PROCEEDINGS (PROTECTION OF PUBLICATION) ACT, 1977 An Act to protect the publication of reports of proceedings of Parliament. [18th April, 1977.]']

# Law GPT Dataset

In [ ]:
dataset1 = load_dataset("nisaar/Lawyer_GPT_India")

Generating train split:   0%|          | 0/150 [00:00<?, ? examples/s]

In [ ]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 150
    })
})

In [ ]:
dataset1['train'][31]['question']

'Can you summarize the main points of the Transgender Persons (Protection of Rights) Act, 2019?'

In [ ]:
dataset1['train'][31]['answer']

'The Transgender Persons Act, 2019 prohibits discrimination against transgender persons in education, employment, healthcare, and other services. It provides for a certificate of identity and establishes a National Council for Transgender Persons. However, critics argue that it does not adequately protect transgender rights and contradicts the NALSA judgment of the Supreme Court.'

# Articles and Constitution QA

In [ ]:
dataset2 = load_dataset("nisaar/Articles_Constitution_3300_Instruction_Set")

Generating train split:   0%|          | 0/3311 [00:00<?, ? examples/s]

In [ ]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt'],
        num_rows: 3311
    })
})

In [ ]:
dataset2['train']['prompt'][9]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDevelop a legal strategy for a hypothetical client based on the facts of the provided case.\n\n### Input:\nYour client is a government company that has terminated the services of a permanent employee without assigning any reason by invoking a similar rule as Rule 9(i) in the Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. case. The terminated employee has challenged the termination order and the validity of the rule in the High Court under Article 226.\n\n### Response:'

In [ ]:
dataset2['train']['output'][9]

"Given the precedent set by the Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. case, your legal strategy should be as follows:1. Reconsider the Termination: Given the Supreme Court's ruling in the precedent case, it would be advisable to reconsider the termination if there is no valid reason for it. If the termination is arbitrary, it is likely to be struck down by the court as violating Article 14.2. Review the Rule: Review the rule under which the termination was made. If it allows for termination without reason, consider revising it to require a valid reason for termination. This would make it less likely to be struck down as arbitrary and violative of Article 14.3. Prepare a Justification: If there is a valid reason for the termination, prepare a detailed justification. This should include evidence of the employee's misconduct or unsatisfactory performance. This will help to defend the termination in court.4. Argue for the Company's Status: While the p

# Analyzing Mistral Tokenizer Chat Format

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
def messages(row: dict) -> list:
    return [{'role': 'user', 'content': '{sample}'.format(sample = row['question'])}, {'role': 'assistant', 'content': '{sample}'.format(sample=row['answer'])}]

In [ ]:
chat_format = tokenizer.apply_chat_template(messages(dataset1['train'][0]), add_generation_prompt=True, return_tensors="pt", tokenize=False)

In [ ]:
chat_format

"<s>[INST] What is the key issue in the legal scenario: 'In Mehta v. Kapoor, Mehta, a tenant, sues Kapoor, a landlord, under the Indian Rent Control Act, for failure to provide basic amenities in the rented property. Mehta faced a blackout due to faulty wiring, which Kapoor knew about.'? [/INST]The key issue in Mehta v. Kapoor is whether the landlord (Kapoor) is liable for the harm to the tenant (Mehta) due to negligence in providing basic amenities, specifically, maintaining the electrical wiring.</s>"

In [ ]:
def messages(row: dict) -> list:
    return [{'role': 'user', 'content': f'{row["instruction"]}\n{row["input"]}'},{'role': 'assistant', 'content': f'{row["output"]}'}]

In [ ]:
chat_format = tokenizer.apply_chat_template(messages(dataset2['train'][0]), return_tensors="pt", tokenize=False)

In [ ]:
chat_format

"<s>[INST] Analyze and explain the legal reasoning behind the judgment in the given case.\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST]The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without reason, conferred unfettered power to terminate employment without he

In [ ]:
del dataset1

In [ ]:
del dataset2

# Dataset preparation for finetuning

In [ ]:
def prompt(context: str, question: str, answer: str) -> list:
    return [
    {
        "role": "user",
        "content": "As a legal consultant and assistant specializing in addressing user queries and drafting legal documents, your role entails providing accurate and practical legal guidance while drafting documents based on the provided context. While answering user queries, ensure relevance and practicality even if the context provided is incomplete. When tasked with drafting legal documents, utilize the format provided in the context and return the document as an HTML file. This file will later be converted to PDF format using a designated function before being delivered to the user. For queries unrelated to document drafting, identify relevant Indian laws, practices, and regulations within the provided context. If insufficient or irrelevant, exercise your knowledge of Indian legal frameworks to provide actionable advice. Avoid speculative content and prioritize realism and practicality."
    },
    {
        "role": "assistant",
        "content": "Absolutely, I'm here to assist you. Based on the details provided, it seems like you need help with legal queries and document drafting. Could you please provide me with the specific context or details of the document you require assistance with?"
    },
    {
        "role": "user",
        "content": "{Context}".format(Context = context)
    },
    {
        "role": "assistant",
        "content": "Thank you for providing the context. Now, what specific question do you have or what assistance do you require regarding this matter?"
    },
    {
        "role": "user",
        "content": "{Question}".format(Question = question)
    },
    {
        "role": "assistant",
        "content": "{Answer}".format(Answer = answer)
    }
]
def messages1(row):
    return {'messages': tokenizer.apply_chat_template(prompt(get_relevant_sentences(row['question'], 2), row['question'], row['answer']), add_generation_prompt=True, return_tensors="pt", tokenize=False)}
def messages2(row):
    return {'messages': tokenizer.apply_chat_template(prompt(get_relevant_sentences(row['input'], 2), row['input'], row['output']), add_generation_prompt=True, return_tensors="pt", tokenize=False)}

In [ ]:
df = pd.concat([pd.DataFrame(load_dataset("nisaar/Lawyer_GPT_India").map(messages1, remove_columns = ['question', 'answer'])['train']), pd.DataFrame(load_dataset("nisaar/Articles_Constitution_3300_Instruction_Set").map(messages2, remove_columns = ['instruction', 'input', 'output', 'prompt'])['train'])], ignore_index = True)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/3311 [00:00<?, ? examples/s]

{TypeError("'NoneType' object is not subscriptable")}


In [ ]:
#checking for the None TYpe Subscritable error cause
df.isnull().sum()

messages    0
dtype: int64

In [ ]:
df.tail(2)
df.to_csv('data.csv')

In [ ]:
vectordb_path='context.bin'
print("Saving index to '%s'" % vectordb_path)
db.save_index(vectordb_path)
del db

Saving index to 'context.bin'


In [ ]:
# saving the Text list to csv for laoding as string in production environment
pd.DataFrame({'Text': Text}).to_csv('Text.csv')
del Text

In [ ]:
# checking the loading of deserialized vector db (downloading the context.bin file and restarting the runtime then loading)
!pip install -q -U sentence-transformers==2.2.2
from sentence_transformers import SentenceTransformer
!pip install -qqq -U --upgrade hnswlib numpy pandas torch
import pandas as pd
import numpy as np
import hnswlib
import torch
db = hnswlib.Index(space = 'l2', dim = 768)
Text = pd.read_csv("Text.csv").Text.tolist()
db.load_index('context.bin', max_elements = len(Text))
device = "cuda" if torch.cuda.is_available() else "cpu"
vector_generator = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)
print([Text[i] for i in db.knn_query(np.array(vector_generator.encode("Right to Vote").tolist()), 2)[0][0]])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 38.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

[' (2) Before giving any decision on any such question, the President shall obtain the opinion of the Election Commission and shall act according to such opinion.]  104. Penalty for sitting and voting before making oath or affirmation under article 99 or when not qualified or when disqualified .—If a person sits or votes as a member of either House of Parliament before he has complied with the requirements of article 99, or when he knows that he is not qualified or that he is disqualifie d for membership thereof, or that he is prohibited from so doing by the provisions of any law made by Parliament, he shall be liable in respect of each day on which he so sits or votes to a penalty of five hundred rupees to be recovered as a debt due to th e Union.  Powers, Privileges and Immunities of Parliament and its Members  105. Powers, privileges, etc., of the Houses of Parliament and of the members and committees thereof. —(1) Subject to the provisions of this Constitution and to the rules and 